In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")
open_ai_model = "gpt-5-nano-2025-08-07"

Enter Your OpenAI API Key:··········


# FLARE

### What is FLARE?

[FLARE](https://arxiv.org/pdf/2305.06983.pdf) stands for `Forward-Looking Active REtrieval` augmented generation.  This is a method that was introduced in 2023 in a paper titled "Active Retrieval Augmented Generation" by Zhengbao Jiang, et al. The original implementation can be found in [the paper's GitHub repo](https://github.com/jzbjyb/FLARE), which the LangChain implementation draws heavy inspiration from.


## 🚀 **Why FLARE Matters**

- Enhances large language models by fetching info from external sources while generating content.

- Tackles the issue of models making stuff up or being factually wrong.

- Aims for outputs that are more accurate and well-informed.

## 🔄 **FLARE's Approach:**
- Starts by answering a question.

- Looks up relevant documents when hitting "uncertain" tokens (based on log probs).

- Uses these documents to keep generating.

- Repeats the process until done.

- Highlights uncertain tokens and forms questions for LLM to find answers.




<img src="https://raw.githubusercontent.com/jzbjyb/FLARE/main/res/flare.gif">

## 🤔 **How FLARE Stands Out from Traditional RAG:**

- Traditional RAG fetches info once based on the input, then generates an output.

- Can be restrictive for long texts.

- FLARE keeps deciding **when and what to fetch** during the generation.

- Predicts the upcoming sentence to guide future content queries.

- If a sentence has low-confidence tokens, it fetches relevant info and rewrites the sentence.


### 💬 **FLARE Prompt Structure:**

- Guides language models to create search queries during answer generation.

- For example: "The colors on the flag of Ghana mean... Red stands for [Search(Ghana flag red meaning)] the blood of martyrs..."

### 🔄 **Direct FLARE Method:**

- Uses the next sentence generated by the language model for retrieval decisions.

- Accepts the sentence if the model is confident.

- If unsure, transforms the sentence into search queries to get more info, then revises the sentence.

### 🛠 **Setting Up the FLARE Chain:**

1. **LLM for Answers**: Needs to return log probabilities to spot uncertain tokens. Recommended: OpenAI wrapper (not ChatOpenAI wrapper, as it doesn't return logprobs).

2. **LLM for Hypothetical Questions**: Can be any model. In this case, we use ChatOpenAI for its speed and cost-effectiveness.

3. **Retriever**: Any retriever works. Here, the SERPER search engine is used for cost efficiency.

#### 🔑 **Key Parameters in FLARE:**

1. **`max_generation_len`**: Caps the number of tokens generated before checking for uncertainties.

2. **`min_prob`**: Tokens generated with a probability below this threshold are marked as uncertain.

In [2]:
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

from langchain_classic.chains import FlareChain

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.callbacks.manager import AsyncCallbackManagerForRetrieverRun, CallbackManagerForRetrieverRun

# We set this so we can see what exactly is going on
from langchain_core.globals import set_verbose

set_verbose(True)

/Users/mo/IADT_local/25_26/AI/Prompt_Engineering_LangChain/Prompt_Engineering_LangChain/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
/Users/mo/IADT_local/25_26/AI/Prompt_Engineering_LangChain/Prompt_Engineering_LangChain/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Same pattern as we saw in the RAG lecture.

In [3]:
# Fix SSL certificate verification (macOS issue)
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


In [10]:
# instantiate llm
llm = ChatOpenAI(logprobs=True) # logprobs=True for FLARE
# logprobs: Whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message.

#instantiate embeddings model
embeddings = OpenAIEmbeddings()

# fetch docs from arxiv, in this case it's the FLARE paper
docs = ArxivLoader(query="2305.06983", load_max_docs=2).load()

# instantiate text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=16)

# split the document into chunks
doc_chunks = text_splitter.split_documents(docs)

# create vector store consisting of embeddigs for document
vector_store = FAISS.from_documents(doc_chunks, embedding=embeddings)

vector_store_retriever = vector_store.as_retriever()

# Set up the FLARE chain

In [11]:
flare = FlareChain.from_llm(
    llm=llm,
    retriever=vector_store_retriever,
    max_generation_len=1024,
    min_prob=0.7,
    verbose=True,
    handle_parsing_errors = True
)

# Start generation

In [12]:
flare.invoke("What is FLARE?")



> Entering new FlareChain chain...
Current Response: 
Generated Questions: ['What are the various contexts in which FLARE can be interpreted?', 'What are some examples of how FLARE can be interpreted or defined?', 'What is FLARE commonly used to describe in the fashion industry?', 'What specific industry might use the term "FLARE"?', 'What could FLARE be used to describe in the fashion industry?', 'Can you provide more information or details in order to help me understand the context better?', 'Can you provide more context so I can give you a _________?']
Current Response:  FLARE stands for Forward-Looking Active Retrieval Augmented Generation. It is a method that achieves superior or competitive performance on long-form knowledge-intensive generation tasks by effectively retrieving necessary information throughout the generation process.
Generated Questions: ['What is FLARE and how does it revolutionize the generation of knowledge-intensive content through active retrieval technique

{'user_input': 'What is FLARE?',
 'response': 'FLARE stands for Forward-Looking Active Retrieval Augmented Generation. It is a method that achieves superior or competitive performance on long-form knowledge-intensive generation tasks by effectively retrieving necessary information throughout the generation process. '}

In [13]:
flare.invoke("What are the steps in a FLARE pipeline?")



> Entering new FlareChain chain...
Current Response: 
Generated Questions: ['What steps typically make up a FLARE pipeline?', 'What are the key steps in a FLARE pipeline that help in assessing the effectiveness and performance of the machine learning solution?', 'What is the significance of the steps in a FLARE pipeline for developing an effective machine learning solution?']
Current Response:  In a FLARE pipeline, the steps typically involve using forward-looking active retrieval augmented generation. This includes retrieving additional information only when the language models do not have the necessary knowledge to avoid unnecessary or inappropriate responses, using reinforcement learning or supervised training where multiple queries can be triggered before generation to improve retrieval quality, and evaluating performance on diverse tasks involving generating long outputs such as multihop QA and commonsense reasoning. The combination of text-based retrievers and potentially a bro

{'user_input': 'What are the steps in a FLARE pipeline?',
 'response': 'In a FLARE pipeline, the steps typically involve using forward-looking active retrieval augmented generation. This includes retrieving additional information only when the language models do not have the necessary knowledge to avoid unnecessary or inappropriate responses, using reinforcement learning or supervised training where multiple queries can be triggered before generation to improve retrieval quality, and evaluating performance on diverse tasks involving generating long outputs such as multihop QA and commonsense reasoning. The combination of text-based retrievers and potentially a browser can enhance the effectiveness of the FLARE method. '}

In [14]:
flare.invoke("What does a typical retrieval instruction look like in FLARE?")



> Entering new FlareChain chain...
Current Response: 
Generated Questions: ['What components are typically included in a retrieval instruction in FLARE?', 'What are some components that a typical retrieval instruction in FLARE may involve specifying?', 'What factors determine the specific format of a retrieval instruction in FLARE?']
Current Response:  In FLARE, a typical retrieval instruction prompts the LM to generate search queries when necessary while generating the answer using retrieval-encouraging instructions, denoted as FLAREinstruct. These instructions guide the model on when and what information to retrieve to enhance the quality of the generated output.
Generated Questions: ['What is involved in a typical retrieval instruction in FLARE?', 'What guides the Language Model (LM) on when and what information to retrieve in FLARE to enhance the quality of the generated output?', 'How does FLARE ensure that generated outputs include essential details by using retrieval instructi

{'user_input': 'What does a typical retrieval instruction look like in FLARE?',
 'response': 'In FLARE, a typical retrieval instruction prompts the LM to generate search queries when necessary while generating the answer using retrieval-encouraging instructions, denoted as FLAREinstruct. These instructions guide the model on when and what information to retrieve to enhance the quality of the generated output. '}